# 🕸️ LangGraph: Orquestrando Agentes como um Maestro Digital!

## Módulo 16 - LangChain v0.3 Course
### Por Pedro Nunes Guth

---

Tá, chegamos aqui no módulo 16 e você já viu **MUITA COISA**: ChatModels, Agents, Tools, RAG... Mas e se eu te disser que existe uma forma de orquestrar tudo isso como se você fosse o maestro de uma orquestra sinfônica? 🎼

É aí que entra o **LangGraph**! Think of it como o "WhatsApp em grupo" dos seus agentes - cada um tem sua função, mas todos trabalham juntos de forma coordenada.

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-16_img_01.png)

## 🤔 Tá, mas o que é LangGraph?

LangGraph é uma extensão do LangChain que permite criar **workflows complexos** usando grafos (graphs). Imagine que você tem vários agentes especializados:

- 🔍 **Agente Pesquisador**: Busca informações na web
- 📊 **Agente Analista**: Processa dados e gera insights
- ✍️ **Agente Escritor**: Cria conteúdo baseado nas análises
- 🔧 **Agente Revisor**: Valida e corrige o resultado final

Com LangGraph, você pode fazer esses agentes conversarem entre si, passarem informações, tomarem decisões e até **voltarem atrás** quando necessário!

### Por que não usar só Chains?

Lembra das Chains que vimos lá no módulo 6? Elas são **lineares** - como uma linha de produção. O LangGraph permite **fluxos condicionais** - como um GPS que recalcula a rota baseado no trânsito!

**Dica!** LangGraph é perfeito quando você precisa de lógica condicional complexa entre seus agentes.

In [ ]:
# Primeiro, vamos instalar tudo que precisamos
!pip install langgraph langchain-google-genai python-dotenv --quiet

# Imports básicos
import os
from dotenv import load_dotenv
from typing import Dict, List, Any
import json

# LangGraph específico
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI

# Carregando variáveis de ambiente
load_dotenv()

print("Libs instaladas! Bora começar! 🚀")

## 🧠 Conceitos Fundamentais

Antes de colocar a mão na massa, vamos entender os conceitos básicos:

### 1. **Nodes (Nós)**
São as "estações" do seu fluxo. Cada node executa uma tarefa específica.

### 2. **Edges (Arestas)**
São as "estradas" que conectam os nodes. Podem ser:
- **Fixas**: Sempre vai para o próximo node
- **Condicionais**: Decide para onde ir baseado no resultado

### 3. **State (Estado)**
É o "WhatsApp do grupo" - todas as informações compartilhadas entre os nodes.

### Analogia Brasileira 🇧🇷
Pense no LangGraph como o **SUS (Sistema Único de Saúde)**:
- Você chega na **recepção** (node inicial)
- Dependendo do seu caso, vai para **clínico geral** ou **especialista** (edges condicionais)
- Cada médico acessa seu **prontuário** (state compartilhado)
- Pode precisar voltar para outros médicos (loops no grafo)

**Dica!** O state é persistente - cada node pode ler e modificar as informações.

In [ ]:
# Vamos criar nosso primeiro exemplo simples
# Configurando o modelo que já conhecemos
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key="Coloca-a-sua-chave-aqui!!!!",
    temperature=0.1
)

# Definindo nossa estrutura de estado
# É como um dicionário que todos os nodes podem acessar
def create_initial_state(user_input: str) -> Dict[str, Any]:
    return {
        "user_input": user_input,
        "research_data": "",
        "analysis": "",
        "final_response": "",
        "step_count": 0
    }

print("Setup básico pronto! 🎯")
print("Agora vamos criar nossos 'funcionários especializados'...")

## 🔍 Criando Nossos Agentes Especializados

Lembra dos Agents que vimos no módulo 11? Agora vamos criar **funções especializadas** que funcionam como mini-agentes dentro do nosso grafo.

Cada função vai:
1. Receber o **state atual**
2. Fazer seu **trabalho específico**
3. Atualizar o **state** com os resultados
4. Retornar o **state modificado**

É como um **revezamento** - cada corredor recebe o bastão, corre sua parte e passa adiante!

In [ ]:
# Node 1: Agente Pesquisador
def research_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Este 'funcionário' é especialista em pesquisa
    Ele pega a pergunta do usuário e simula uma pesquisa
    """
    print("🔍 Agente Pesquisador trabalhando...")

    user_question = state["user_input"]

    # Prompt para simular pesquisa
    research_prompt = f"""
    Você é um pesquisador expert. Sua tarefa é simular uma pesquisa sobre: {user_question}

    Forneça dados e informações relevantes como se você tivesse pesquisado em fontes confiáveis.
    Seja específico e inclua números, estatísticas ou fatos quando possível.

    Formato: Lista com 3-5 pontos principais de pesquisa.
    """

    # Fazendo a pesquisa com nossa LLM
    research_result = llm.invoke(research_prompt)

    # Atualizando o state
    state["research_data"] = research_result.content
    state["step_count"] += 1

    print(f"✅ Pesquisa concluída! Step {state['step_count']}")
    return state

# Node 2: Agente Analista
def analysis_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Este 'funcionário' pega os dados da pesquisa e faz análise
    """
    print("📊 Agente Analista trabalhando...")

    research_data = state["research_data"]
    original_question = state["user_input"]

    analysis_prompt = f"""
    Você é um analista expert. Baseado nestes dados de pesquisa:

    {research_data}

    Faça uma análise crítica e inteligente relacionada à pergunta original: {original_question}

    Inclua:
    - Insights principais
    - Padrões identificados
    - Conclusões baseadas nos dados
    """

    analysis_result = llm.invoke(analysis_prompt)

    state["analysis"] = analysis_result.content
    state["step_count"] += 1

    print(f"✅ Análise concluída! Step {state['step_count']}")
    return state

print("Agentes criados! 🤖")
print("Próximo: vamos criar o agente que junta tudo...")

In [ ]:
# Node 3: Agente Sintetizador (junta tudo)
def synthesis_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Este é o 'chefe' que junta o trabalho de todos
    """
    print("✍️ Agente Sintetizador trabalhando...")

    user_question = state["user_input"]
    research = state["research_data"]
    analysis = state["analysis"]

    synthesis_prompt = f"""
    Você é um sintetizador expert. Sua missão é criar uma resposta final completa e útil.

    PERGUNTA ORIGINAL: {user_question}

    DADOS DE PESQUISA:
    {research}

    ANÁLISE REALIZADA:
    {analysis}

    Crie uma resposta final que:
    1. Responda diretamente à pergunta
    2. Use os dados pesquisados
    3. Inclua os insights da análise
    4. Seja clara e acionável

    Tom: Profissional mas acessível, como o Pedro Guth explicaria! 🚀
    """

    final_result = llm.invoke(synthesis_prompt)

    state["final_response"] = final_result.content
    state["step_count"] += 1

    print(f"✅ Síntese concluída! Step {state['step_count']}")
    print("🎉 Workflow completo!")
    return state

# Função para decidir qual caminho seguir (edge condicional)
def decide_next_step(state: Dict[str, Any]) -> str:
    """
    Esta função decide para onde ir baseado no estado atual
    É como um semáforo inteligente!
    """
    step_count = state["step_count"]

    if step_count == 1:  # Acabou de pesquisar
        return "analysis"
    elif step_count == 2:  # Acabou de analisar
        return "synthesis"
    else:  # Acabou tudo
        return END

print("Todas as funções criadas! 🛠️")
print("Agora vamos montar o grafo...")

## 🕸️ Montando o Grafo

Agora vem a parte **MAIS LEGAL**! Vamos conectar nossos agentes como se fossem peças de Lego:

```
USUÁRIO → PESQUISA → ANÁLISE → SÍNTESE → RESPOSTA FINAL
```

O LangGraph vai cuidar de:
- ✅ Executar cada node na ordem certa
- ✅ Passar o state entre eles
- ✅ Tomar decisões baseado nas condições
- ✅ Gerenciar todo o fluxo automaticamente

É como ter um **assistente pessoal digital** coordenando uma equipe inteira!

In [ ]:
# Criando o grafo - É aqui que a mágica acontece! ✨
workflow = StateGraph(dict)

# Adicionando nossos 'funcionários' (nodes)
workflow.add_node("research", research_node)
workflow.add_node("analysis", analysis_node)
workflow.add_node("synthesis", synthesis_node)

# Definindo o ponto de entrada (onde tudo começa)
workflow.set_entry_point("research")

# Conectando os nodes com edges condicionais
workflow.add_conditional_edges(
    "research",
    decide_next_step,
    {
        "analysis": "analysis",
        END: END
    }
)

workflow.add_conditional_edges(
    "analysis",
    decide_next_step,
    {
        "synthesis": "synthesis",
        END: END
    }
)

workflow.add_conditional_edges(
    "synthesis",
    decide_next_step,
    {
        END: END
    }
)

# Compilando o grafo (transformando em uma aplicação executável)
app = workflow.compile()

print("🎯 Grafo montado e compilado!")
print("Agora temos um sistema completo de IA working together!")

## 🚀 Executando Nosso Primeiro Workflow

Chegou a hora da verdade! Vamos fazer nosso "time de especialistas digitais" trabalhar junto.

É como dar o play em uma sinfonia - cada instrumento (agente) vai tocar sua parte no momento certo!

In [ ]:
# Teste prático - Vamos fazer uma pergunta!
user_question = "Quais são as principais tendências de IA para 2025?"

print(f"🤔 Pergunta do usuário: {user_question}")
print("\n" + "="*50)
print("🚀 INICIANDO WORKFLOW LANGGRAPH")
print("="*50 + "\n")

# Criando o estado inicial
initial_state = create_initial_state(user_question)

# Executando o workflow completo!
# O LangGraph vai cuidar de tudo automaticamente
final_state = app.invoke(initial_state)

print("\n" + "="*50)
print("✅ WORKFLOW CONCLUÍDO!")
print("="*50)

In [ ]:
# Vamos ver o que cada agente produziu!
print("📋 RELATÓRIO COMPLETO DO WORKFLOW:\n")

print("🔍 DADOS DA PESQUISA:")
print("-" * 30)
print(final_state["research_data"][:300] + "...")

print("\n📊 ANÁLISE REALIZADA:")
print("-" * 30)
print(final_state["analysis"][:300] + "...")

print("\n🎯 RESPOSTA FINAL:")
print("-" * 30)
print(final_state["final_response"])

print(f"\n📈 Total de steps executados: {final_state['step_count']}")

## 📊 Visualizando o Fluxo

Uma das coisas **MAIS LINDAS** do LangGraph é que podemos visualizar nosso workflow! É como ter um mapa do pensamento dos nossos agentes.

Vamos criar um diagrama simples para entender o fluxo:

In [ ]:
# Vamos criar uma visualização simples do nosso workflow
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch
import numpy as np

fig, ax = plt.subplots(figsize=(12, 8))

# Definindo posições dos nodes
positions = {
    'START': (1, 4),
    'RESEARCH': (3, 4),
    'ANALYSIS': (5, 4),
    'SYNTHESIS': (7, 4),
    'END': (9, 4)
}

# Cores para cada tipo de node
colors = {
    'START': '#90EE90',      # Verde claro
    'RESEARCH': '#87CEEB',   # Azul claro
    'ANALYSIS': '#DDA0DD',   # Roxo claro
    'SYNTHESIS': '#F0E68C',  # Amarelo claro
    'END': '#FFB6C1'        # Rosa claro
}

# Desenhando os nodes
for node, (x, y) in positions.items():
    # Criando caixas arredondadas
    bbox = FancyBboxPatch(
        (x-0.4, y-0.3), 0.8, 0.6,
        boxstyle="round,pad=0.1",
        facecolor=colors[node],
        edgecolor='black',
        linewidth=2
    )
    ax.add_patch(bbox)

    # Adicionando texto
    ax.text(x, y, node, ha='center', va='center', fontsize=10, fontweight='bold')

# Desenhando as setas (edges)
arrows = [
    ((1.4, 4), (2.6, 4)),   # START -> RESEARCH
    ((3.4, 4), (4.6, 4)),   # RESEARCH -> ANALYSIS
    ((5.4, 4), (6.6, 4)),   # ANALYSIS -> SYNTHESIS
    ((7.4, 4), (8.6, 4))    # SYNTHESIS -> END
]

for (x1, y1), (x2, y2) in arrows:
    ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                arrowprops=dict(arrowstyle='->', lw=2, color='darkblue'))

# Configurando o gráfico
ax.set_xlim(0, 10)
ax.set_ylim(3, 5)
ax.set_aspect('equal')
ax.axis('off')

plt.title('🕸️ LangGraph Workflow - Fluxo dos Agentes', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("🎨 Liiindo! Esse é o mapa do nosso workflow!")
print("Cada caixa é um agente especializado trabalhando em sequência.")

## 🔄 Workflows com Loops e Condições

Agora vamos para a parte **AVANÇADA**! E se nosso agente precisar **voltar** e refazer algo? Ou tomar decisões diferentes baseado no conteúdo?

Vamos criar um workflow que pode:
- ✅ **Validar** se a resposta está boa
- ✅ **Voltar** para melhorar se necessário
- ✅ **Decidir** caminhos diferentes baseado no conteúdo

É como ter um **editor chato** que sempre pede para melhorar o texto! 😅

**Dica!** Isso é o que diferencia LangGraph das Chains simples - a capacidade de ter loops e condições complexas.

In [ ]:
# Vamos criar um workflow mais avançado com validação

def content_generator_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """Gera conteúdo inicial"""
    print("✍️ Gerando conteúdo...")

    topic = state["topic"]
    attempt = state.get("attempt", 1)

    if attempt > 1:
        feedback = state.get("feedback", "")
        prompt = f"""
        Reescreva o conteúdo sobre: {topic}

        Feedback para melhorar: {feedback}

        Tentativa #{attempt} - Faça melhor desta vez!
        """
    else:
        prompt = f"""
        Escreva um conteúdo interessante sobre: {topic}

        Faça algo informativo e engajante.
        """

    result = llm.invoke(prompt)

    state["content"] = result.content
    state["attempt"] = attempt

    return state

def validator_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """Valida se o conteúdo está bom"""
    print("🔍 Validando conteúdo...")

    content = state["content"]

    validation_prompt = f"""
    Analise este conteúdo e diga se está bom ou precisa melhorar:

    {content}

    Critérios:
    - Está informativo?
    - Está bem estruturado?
    - Tem pelo menos 100 palavras?

    Responda apenas:
    "APROVADO" se está bom
    "REPROVAR: [motivo]" se precisa melhorar
    """

    validation = llm.invoke(validation_prompt)

    state["validation_result"] = validation.content

    if "REPROVAR" in validation.content:
        state["feedback"] = validation.content.replace("REPROVAR:", "").strip()
        state["needs_improvement"] = True
    else:
        state["needs_improvement"] = False

    return state

def decide_improvement(state: Dict[str, Any]) -> str:
    """Decide se precisa melhorar ou se está pronto"""
    max_attempts = 3
    current_attempt = state.get("attempt", 1)

    if state.get("needs_improvement", False) and current_attempt < max_attempts:
        # Incrementa tentativa e volta para gerar conteúdo
        state["attempt"] = current_attempt + 1
        return "generate"  # Volta para o gerador
    else:
        return END  # Termina (ou está aprovado, ou atingiu limite)

print("🔄 Workflow avançado com loops criado!")

In [ ]:
# Montando o workflow avançado
advanced_workflow = StateGraph(dict)

# Adicionando nodes
advanced_workflow.add_node("generate", content_generator_node)
advanced_workflow.add_node("validate", validator_node)

# Entry point
advanced_workflow.set_entry_point("generate")

# Connections
# Gerador sempre vai para validador
advanced_workflow.add_edge("generate", "validate")

# Validador decide se volta para gerador ou termina
advanced_workflow.add_conditional_edges(
    "validate",
    decide_improvement,
    {
        "generate": "generate",  # Loop de volta!
        END: END
    }
)

# Compilando
advanced_app = advanced_workflow.compile()

print("🎯 Workflow avançado compilado!")
print("Este pode fazer loops até ficar perfeito! 🔄")

In [ ]:
# Testando o workflow avançado
topic = "Benefícios da IA na educação brasileira em menos de 99 palavras"

advanced_state = {
    "topic": topic,
    "attempt": 1,
    "content": "",
    "validation_result": "",
    "feedback": "",
    "needs_improvement": False
}

print(f"🎯 Tópico: {topic}")
print("\n🚀 Executando workflow com validação...\n")

# Executando
final_advanced_state = advanced_app.invoke(advanced_state)

print("\n" + "="*50)
print("📊 RESULTADO FINAL:")
print("="*50)
print(f"Tentativas: {final_advanced_state['attempt']}")
print(f"Validação: {final_advanced_state['validation_result']}")
print("\nConteúdo final:")
print("-" * 30)
print(final_advanced_state['content'])

## 🎯 Comparando com o que Já Sabemos

Vamos fazer uma **comparação prática** entre as abordagens que já conhecemos:

### 🔗 Chains (Módulo 6)
- ✅ **Simples** e direto
- ❌ **Linear** - não pode voltar
- ❌ **Sem condições** complexas

### 🤖 Agents (Módulo 11)
- ✅ **Inteligente** - usa tools
- ✅ **Flexível** - decide ações
- ❌ **Um agente só** - não coordena múltiplos

### 🕸️ LangGraph (AGORA!)
- ✅ **Múltiplos agentes** especializados
- ✅ **Fluxos condicionais** complexos
- ✅ **Loops** e validações
- ✅ **Estado compartilhado** entre todos
- ✅ **Orquestração** inteligente

**Dica!** Use LangGraph quando precisar de workflows complexos com múltiplos "especialistas" trabalhando juntos.

In [ ]:
# Vamos comparar na prática - mesma tarefa, abordagens diferentes
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

question = "Como implementar IA no atendimento ao cliente?"

print("🥊 COMPARAÇÃO PRÁTICA: Chain vs LangGraph")
print("="*50)

# Abordagem 1: Chain simples (que já conhecemos)
print("\n🔗 ABORDAGEM 1: CHAIN SIMPLES")
print("-" * 30)

simple_prompt = PromptTemplate(
    input_variables=["question"],
    template="Responda esta pergunta de forma completa: {question}"
)

simple_chain = simple_prompt | llm
chain_result = simple_chain.invoke({"question": question})

print("Resultado Chain:")
print(chain_result.content[:200] + "...")

# Abordagem 2: LangGraph (nosso workflow)
print("\n🕸️ ABORDAGEM 2: LANGGRAPH")
print("-" * 30)

graph_state = create_initial_state(question)
graph_result = app.invoke(graph_state)

print("Resultado LangGraph:")
print(graph_result["final_response"][:200] + "...")

print("\n📊 DIFERENÇAS:")
print("Chain: 1 etapa, resposta direta")
print("LangGraph: 3 etapas (pesquisa + análise + síntese)")
print("\nLangGraph é mais rico e estruturado! 🚀")

## 🛠️ Casos de Uso Reais

Agora que você entendeu o conceito, vamos ver onde o LangGraph **brilha** na vida real:

### 1. 🏥 **Diagnóstico Médico Assistido**
- Agente coleta sintomas
- Agente pesquisa condições possíveis
- Agente analisa exames
- Agente gera relatório para médico

### 2. 📈 **Análise Financeira Completa**
- Agente coleta dados do mercado
- Agente analisa tendências
- Agente calcula riscos
- Agente gera recomendações

### 3. 🎓 **Tutor Educacional Personalizado**
- Agente avalia nível do aluno
- Agente adapta conteúdo
- Agente monitora progresso
- Agente ajusta estratégia

### 4. 🏢 **Assistente Empresarial Completo**
- Agente analisa documentos
- Agente pesquisa regulamentações
- Agente gera relatórios
- Agente sugere ações

**Dica!** LangGraph é perfeito para qualquer tarefa que normalmente precisaria de uma "equipe" de especialistas.

## 💡 Exercício Prático

**DESAFIO**: Crie um workflow LangGraph para análise de currículo!

Seu workflow deve ter:
1. **Agente Extrator**: Extrai informações principais do currículo
2. **Agente Avaliador**: Avalia pontos fortes e fracos
3. **Agente Conselheiro**: Dá sugestões de melhoria

**Entrada**: Texto de um currículo
**Saída**: Análise completa com sugestões

Vamos fazer juntos! 🚀

In [ ]:
# EXERCÍCIO: Complete o código abaixo!

def extract_info_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """SEU CÓDIGO AQUI: Extrair informações do currículo"""
    print("📋 Extraindo informações do currículo...")

    curriculum = state["curriculum_text"]

    # DICA: Crie um prompt para extrair:
    # - Experiências
    # - Habilidades
    # - Formação
    # - Contato

    extract_prompt = f"""
    Extraia as principais informações deste currículo:

    {curriculum}

    Organize em:
    - EXPERIÊNCIAS:
    - HABILIDADES:
    - FORMAÇÃO:
    - CONTATO:
    """

    result = llm.invoke(extract_prompt)
    state["extracted_info"] = result.content

    return state

def evaluate_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """SEU CÓDIGO AQUI: Avaliar pontos fortes e fracos"""
    print("⚖️ Avaliando currículo...")

    # COMPLETE AQUI!
    extracted = state["extracted_info"]

    evaluate_prompt = f"""
    Baseado nestas informações extraídas:

    {extracted}

    Avalie:
    PONTOS FORTES:
    PONTOS FRACOS:
    NOTA GERAL (1-10):
    """

    result = llm.invoke(evaluate_prompt)
    state["evaluation"] = result.content

    return state

def advice_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """SEU CÓDIGO AQUI: Dar conselhos de melhoria"""
    print("💡 Gerando conselhos...")

    # COMPLETE AQUI!
    evaluation = state["evaluation"]

    advice_prompt = f"""
    Baseado nesta avaliação:

    {evaluation}

    Dê 5 conselhos práticos para melhorar o currículo:

    1.
    2.
    3.
    4.
    5.
    """

    result = llm.invoke(advice_prompt)
    state["advice"] = result.content

    return state

print("📝 Exercício preparado! Complete as funções acima.")

In [ ]:
# Montando o workflow do exercício
curriculum_workflow = StateGraph(dict)

# Adicionando nodes
curriculum_workflow.add_node("extract", extract_info_node)
curriculum_workflow.add_node("evaluate", evaluate_node)
curriculum_workflow.add_node("advice", advice_node)

# Entry point
curriculum_workflow.set_entry_point("extract")

# Edges sequenciais
curriculum_workflow.add_edge("extract", "evaluate")
curriculum_workflow.add_edge("evaluate", "advice")
curriculum_workflow.add_edge("advice", END)

# Compilando
curriculum_app = curriculum_workflow.compile()

print("✅ Workflow de análise de currículo criado!")

In [ ]:
# Testando com um currículo exemplo
sample_curriculum = """
João Silva
Desenvolvedor Python
Email: joao@email.com
Telefone: (11) 99999-9999

EXPERIÊNCIA:
- Desenvolvedor Jr na TechCorp (2022-2024)
- Estagiário na StartupX (2021-2022)

HABILIDADES:
- Python, Django, Flask
- HTML, CSS, JavaScript
- Git, Docker

FORMAÇÃO:
- Ciência da Computação - UNIFEI (2020-2024)
"""

# Estado inicial
curriculum_state = {
    "curriculum_text": sample_curriculum,
    "extracted_info": "",
    "evaluation": "",
    "advice": ""
}

print("🎯 Analisando currículo do João...\n")

# Executando
final_analysis = curriculum_app.invoke(curriculum_state)

print("\n" + "="*50)
print("📊 ANÁLISE COMPLETA DE CURRÍCULO")
print("="*50)

print("\n📋 INFORMAÇÕES EXTRAÍDAS:")
print(final_analysis["extracted_info"])

print("\n⚖️ AVALIAÇÃO:")
print(final_analysis["evaluation"])

print("\n💡 CONSELHOS:")
print(final_analysis["advice"])

print("\n🎉 Parabéns! Você criou um sistema completo de análise!")

## 🚀 Indo Além: Integrando com Tools

Lembra das **Tools** que vimos no módulo 11? Podemos integrar elas com LangGraph!

Imagine um agente que:
1. **Pesquisa** na web (usando tool de busca)
2. **Calcula** valores (usando tool de matemática)
3. **Envia** emails (usando tool de email)
4. **Salva** em banco de dados (usando tool de BD)

É como ter **super poderes** em cada node do grafo!

In [ ]:
# Exemplo rápido: LangGraph + Tools
from langchain.tools import Tool

# Criando uma tool simples (lembra do módulo 11?)
def calculate_roi(investment: str) -> str:
    """Calcula ROI básico"""
    try:
        # Parsing simples - na vida real seria mais robusto
        parts = investment.split(",")
        initial = float(parts[0])
        final = float(parts[1])
        roi = ((final - initial) / initial) * 100
        return f"ROI: {roi:.2f}%"
    except:
        return "Erro no cálculo. Use formato: valor_inicial,valor_final"

# Criando a tool
roi_tool = Tool(
    name="calculate_roi",
    description="Calcula ROI. Input: 'valor_inicial,valor_final'",
    func=calculate_roi
)

def investment_analyzer_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """Node que usa tools para análise de investimento"""
    print("💰 Analisando investimento com tools...")

    # Simulando uso de tool
    investment_data = "1000,1200"  # R$ 1000 virou R$ 1200
    roi_result = roi_tool.func(investment_data)

    state["roi_calculation"] = roi_result

    # Agora usa LLM para interpretar
    interpretation_prompt = f"""
    Baseado neste cálculo de ROI: {roi_result}

    Dê uma interpretação:
    - É um bom retorno?
    - Qual o risco?
    - Recomendações?
    """

    analysis = llm.invoke(interpretation_prompt)
    state["investment_analysis"] = analysis.content

    return state

# Testando
investment_state = {"roi_calculation": "", "investment_analysis": ""}
result = investment_analyzer_node(investment_state)

print("\n📊 RESULTADO:")
print(f"Cálculo: {result['roi_calculation']}")
print(f"Análise: {result['investment_analysis'][:150]}...")

print("\n🎯 ISSO é o poder do LangGraph + Tools!")
print("Cada node pode usar ferramentas externas!")

## 📈 Performance e Otimização

Quando você começa a usar LangGraph em **produção**, algumas dicas importantes:

### 🚀 **Dicas de Performance**

1. **Paralelização**: Alguns nodes podem rodar em paralelo
2. **Cache**: Evite recalcular coisas iguais
3. **Timeouts**: Defina limites para evitar travamentos
4. **Logging**: Monitore cada step do workflow

### 💡 **Boas Práticas**

- ✅ **Nodes pequenos** e focados
- ✅ **Estado mínimo** - só o necessário
- ✅ **Tratamento de erros** em cada node
- ✅ **Documentação** clara do fluxo

**Dica!** Em produção, sempre monitore o tempo de execução de cada node!

In [ ]:
# Exemplo de monitoramento simples
import time
from datetime import datetime

def monitored_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """Node com monitoramento básico"""
    start_time = time.time()

    print(f"⏰ [{datetime.now().strftime('%H:%M:%S')}] Iniciando node...")

    try:
        # Simulando processamento
        time.sleep(1)  # Simula 1 segundo de trabalho

        state["result"] = "Processamento concluído com sucesso!"
        state["status"] = "success"

    except Exception as e:
        print(f"❌ Erro: {str(e)}")
        state["result"] = f"Erro: {str(e)}"
        state["status"] = "error"

    finally:
        end_time = time.time()
        execution_time = end_time - start_time

        print(f"⏱️ Node executado em {execution_time:.2f}s")
        state["execution_time"] = execution_time

    return state

# Testando monitoramento
test_state = {}
monitored_result = monitored_node(test_state)

print("\n📊 Estado final:")
for key, value in monitored_result.items():
    print(f"{key}: {value}")

print("\n🎯 Em produção, você salvaria esses logs em um sistema de monitoramento!")

## 🎭 Casos de Uso Avançados

Para finalizar, vamos ver alguns casos **SUPER AVANÇADOS** onde LangGraph realmente brilha:

### 1. **Sistema de Aprovação Multi-Nível**
```
SOLICITAÇÃO → SUPERVISOR → GERENTE → DIRETOR → APROVADO/NEGADO
     ↑______________|_______________|____________↑
          (pode voltar para qualquer nível)
```

### 2. **Chatbot com Contexto Complexo**
```
USUÁRIO → CLASSIFICADOR → ESPECIALISTA_A → VALIDADOR → RESPOSTA
              ↓              ↑              ↑
         ESPECIALISTA_B → COMBINADOR ------↑
```

### 3. **Pipeline de ML Completo**
```
DADOS → LIMPEZA → FEATURE_ENG → TREINO → VALIDAÇÃO → DEPLOY
   ↑        ↓          ↑           ↓        ↓         ↑
   └────FEEDBACK──────────────────────────┘─────────┘
```

**Dica!** LangGraph permite modelar qualquer processo complexo que você imaginar!

## 🎯 Resumo: O que Aprendemos

**Parabéns!** 🎉 Você dominou o LangGraph! Aqui está o que cobrimos:

### ✅ **Conceitos Fundamentais**
- Nodes, Edges e State
- Diferenças entre Chains, Agents e LangGraph
- Fluxos condicionais e loops

### ✅ **Implementação Prática**
- Criamos workflows completos
- Integramos múltiplos agentes especializados
- Implementamos validação e melhoria iterativa

### ✅ **Casos de Uso Reais**
- Análise de conteúdo com validação
- Análise de currículo multi-etapas
- Integração com Tools externas

### ✅ **Boas Práticas**
- Monitoramento e logging
- Tratamento de erros
- Otimização de performance

### 🚀 **Próximos Passos**
No **módulo 17**, vamos ver o **LangSmith** - a ferramenta para monitorar, debuggar e otimizar tudo que construímos!

**Dica Final!** LangGraph é perfeito para substituir sistemas complexos que normalmente precisariam de múltiplas pessoas ou ferramentas separadas. É IA trabalhando em equipe! 🤝

## 🏆 Desafio Final

**MISSÃO ESPECIAL**: Crie um LangGraph que simula uma **consultoria empresarial completa**!

### Requisitos:
1. **Agente Diagnóstico**: Analisa problemas da empresa
2. **Agente Pesquisador**: Busca soluções similares no mercado
3. **Agente Estrategista**: Propõe estratégias
4. **Agente Validador**: Valida viabilidade das propostas
5. **Agente Apresentador**: Cria apresentação final

### Diferenciais:
- ✅ Loops de validação
- ✅ Decisões condicionais baseadas no tipo de empresa
- ✅ Integração com tools (se possível)
- ✅ Monitoramento de cada etapa

**Prazo**: Próxima aula! 😄

---

### 🔥 **Você chegou até aqui = VOCÊ É TOP!**

LangGraph é uma das ferramentas mais avançadas do ecossistema LangChain. Dominando ela, você pode criar sistemas de IA **extremamente sofisticados**.

**Nos vemos no módulo 17** para descobrir como monitorar e otimizar tudo isso com **LangSmith**! 🚀

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-16_img_02.png)